In [1]:
import pandas_datareader.data as web
from fredapi import Fred

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from numpy import hstack
from scipy import stats
from datetime import datetime
from sklearn.preprocessing import StandardScaler

import warnings
import os

warnings.filterwarnings('ignore')

import itertools
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import plot_pacf
from statsmodels.graphics.tsaplots import plot_acf
import pmdarima as pm
from pmdarima import model_selection

from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import TimeDistributed
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D

%matplotlib inline

Using TensorFlow backend.


In [140]:
start = datetime(2000,2,1)
end = datetime(2020,1,31)

## Exogenous Data Gathering

In [141]:
fred = Fred(api_key = 'aec5f13aa40d1af4edd587ffccceeb1a')

In [142]:
# LIBOR rate (USD1MTD156N)
usd1mtd156n = fred.get_series('usd1mtd156n', start, end)
usd1mtd156n = round(usd1mtd156n.resample('MS').mean(),2)
usd1mtd156n = pd.DataFrame(usd1mtd156n, index = usd1mtd156n.index, columns = ['usd1mtd156n'])

In [143]:
# 30 Year Fixed Rate Mortgage (MORTGAGE30US)
mortgage30us = fred.get_series('mortgage30us', start, end)
mortgage30us = round(mortgage30us.resample('MS').mean(),2)
mortgage30us = pd.DataFrame(mortgage30us, index = mortgage30.index, columns = ['mortgage30us'])

In [144]:
# 15 Year Fixed rate Mortgage (MORTGAGE15US)
mortgage15us = fred.get_series('MORTGAGE15US', start, end)
mortgage15us = round(mortgage15us.resample('MS').mean(),2)
mortgage15us = pd.DataFrame(mortgage15us, index = mortgage15.index, columns = ['mortgage15us'])

In [145]:
# 3-Month AA Financial Commercial Paper Rate (DCPF3M)
dcpf3m = fred.get_series('DCPF3M', start, end)
dcpf3m = round(dcpf3m.resample('MS').mean(),2)
dcpf3m = pd.DataFrame(dcpf3m, index = dcpf3m.index, columns = ['dcpf3m'])

In [146]:
# 3-Month AA Nonfinancial Commercial Paper Rate (CPN3M)
cpn3m = fred.get_series('cpn3m', start, end)
cpn3m = round(cpn3m.resample('MS').mean(),2)
cpn3m = pd.DataFrame(cpn3m, index = dcpf3m.index, columns = ['cpn3m'])

In [147]:
# 1-Month AA Financial Commercial Paper Rate (DCPF1M)
dcpf1m = fred.get_series('dcpf1m', start, end)
dcpf1m = round(dcpf1m.resample('MS').mean(),2)
dcpf1m = pd.DataFrame(dcpf1m, index = dcpf3m.index, columns = ['dcpf1m'])

In [148]:
# Moody's Seasoned Aaa Corporate Bond Yield (AAA)
aaa = fred.get_series('AAA', start, end)
aaa = round(aaa.resample('MS').mean(),2)
aaa = pd.DataFrame(aaa, index = dcpf3m.index, columns = ['aaa'])

In [149]:
# Moody's Seasoned Baa Corporate Bond Yield (BAA)
baa = fred.get_series('BAA', start, end)
baa = round(baa.resample('MS').mean(),2)
baa = pd.DataFrame(baa, index = baa.index, columns = ['baa'])

In [150]:
# Effective Federal Funds Rate (DFF)
dff = fred.get_series('DFF', start, end)
dff = round(dff.resample('MS').mean(),2)
dff = pd.DataFrame(dff, index = dff.index, columns = ['dff'])

In [151]:
# Eighth District Flexible Rate on Seasonal Credit (FLEXSC)
flexsc = fred.get_series('flexsc', start, end)
flexsc = round(flexsc.resample('MS').mean(),2)
flexsc = pd.DataFrame(flexsc, index = flexsc.index, columns = ['flexsc'])

In [152]:
# 10-Year Treasury Constant Maturity Minus 2-Year Treasury Constant Maturity (T10Y2Y)
t10y2y = fred.get_series('t10y2y', start, end)
t10y2y = round(t10y2y.resample('MS').mean(),2)
t10y2y = pd.DataFrame(t10y2y, index = t10y2y.index, columns = ['t10y2y'])

In [153]:
# 10-Year Treasury Constant Maturity Minus 3-Month Treasury Constant Maturity (T10Y3M)
t10y3m = fred.get_series('t10y3m', start, end)
t10y3m = round(t10y3m.resample('MS').mean(),2)
t10y3m = pd.DataFrame(t10y3m, index = t10y3m.index, columns = ['t10y3m'])

In [154]:
# TED Spread (TEDRATE)
tedrate = fred.get_series('tedrate', start, end)
tedrate = round(tedrate.resample('MS').mean(),2)
tedrate = pd.DataFrame(tedrate, index = tedrate.index, columns = ['tedrate'])

In [160]:
# 10-Year Treasury Constant Maturity Minus Federal Funds Rate (T10YFF)
t10yff = fred.get_series('T10YFF', start, end)
t10yff = round(t10yff.resample('MS').mean(),2)
t10yff = pd.DataFrame(t10yff, index = t10yff.index, columns = ['t10yff'])

In [162]:
# 5-Year Treasury Constant Maturity Minus Federal Funds Rate (T5YFF)
t5yff = fred.get_series('t5yff', start, end)
t5yff = round(t5yff.resample('MS').mean(),2)
t5yff = pd.DataFrame(t5yff, index = t5yff.index, columns = ['t5yff'])

In [166]:
# 6-Month Treasury Bill Minus Federal Funds Rate (TB6SMFFM)
tb6smffm = fred.get_series('tb6smffm', start, end)
tb6smffm = round(tb6smffm.resample('MS').mean(),2)
tb6smffm = pd.DataFrame(tb6smffm, index = tb6smffm.index, columns = ['tb6smffm'])

In [176]:
# 3-Month London Interbank Offered Rate (LIBOR), based on U.S. Dollar (USD3MTD156N)
usd3mtd156n = fred.get_series('usd3mtd156n', start, end)
usd3mtd156n = round(usd3mtd156n.resample('MS').mean(),2)
usd3mtd156n = pd.DataFrame(usd3mtd156n, index = usd3mtd156n.index, columns = ['usd3mtd156n'])

In [178]:
# 12-Month London Interbank Offered Rate (LIBOR), based on U.S. Dollar (USD12MD156N)
usd12md156n = fred.get_series('usd12md156n', start, end)
usd12md156n = round(usd12md156n.resample('MS').mean(),2)
usd12md156n = pd.DataFrame(usd12md156n, index = usd12md156n.index, columns = ['usd12md156n'])

In [182]:
# 1-Week London Interbank Offered Rate (LIBOR), based on Euro (EUR1WKD156N)
eur1wkd156n = fred.get_series('eur1wkd156n', start, end)
eur1wkd156n = round(eur1wkd156n.resample('MS').mean(),2)
eur1wkd156n = pd.DataFrame(eur1wkd156n, index = eur1wkd156n.index, columns = ['eur1wkd156n'])

In [183]:
# Bank Prime Loan Rate (MPRIME)
mprime = fred.get_series('mprime', start, end)
mprime = round(mprime.resample('MS').mean(),2)
mprime = pd.DataFrame(mprime, index = mprime.index, columns = ['mprime'])

In [185]:
# 3-Month Treasury Bill: Secondary Market Rate (TB3MS)
tb3ms = fred.get_series('tb3ms', start, end)
tb3ms = round(tb3ms.resample('MS').mean(),2)
tb3ms = pd.DataFrame(tb3ms, index = tb3ms.index, columns = ['tb3ms'])

In [187]:
# 1-Year Treasury Bill: Secondary Market Rate (TB1YR)
tb1yr = fred.get_series('tb1yr', start, end)
tb1yr = round(tb1yr.resample('MS').mean(),2)
tb1yr = pd.DataFrame(tb1yr, index = tb1yr.index, columns = ['tb1yr'])

In [191]:
# 6-Month Treasury Bill: Secondary Market Rate (DTB6)
dtb6 = fred.get_series('dtb6', start, end)
dtb6 = round(dtb6.resample('MS').mean(),2)
dtb6= pd.DataFrame(dtb6, index = dtb6.index, columns = ['dtb6'])

In [196]:
# 10-Year Treasury Constant Maturity Rate (DGS10)
dgs10 = fred.get_series('dgs10', start, end)
dgs10 = round(dgs10.resample('MS').mean(),2)
dgs10 = pd.DataFrame(dgs10, index = dgs10.index, columns = ['dgs10'])

In [198]:
# 1-Year Treasury Constant Maturity Rate (DGS1)
dgs1 = fred.get_series('dgs1', start, end)
dgs1 = round(dgs1.resample('MS').mean(),2)
dgs1 = pd.DataFrame(dgs1, index = dgs1.index, columns = ['dgs1'])

In [200]:
# 10-Year Treasury Inflation-Indexed Security, Constant Maturity (DFII10)
dfii10 = fred.get_series('dfii10', start, end)
dfii10 = round(dfii10.resample('MS').mean(),2)
dfii10 = pd.DataFrame(dfii10, index = dfii10.index, columns = ['dfii10'])

In [204]:
# 3-Month Treasury Constant Maturity Rate (DGS3MO)
dgs3mo = fred.get_series('dgs3mo', start, end)
dgs3mo = round(dgs3mo.resample('MS').mean(),2)
dgs3mo = pd.DataFrame(dgs3mo, index = dgs3mo.index, columns = ['dgs3mo'])

In [208]:
# 30-Year Treasury Constant Maturity Rate (DGS30)
dgs30 = fred.get_series('dgs30', start, end)
dgs30 = round(dgs30.resample('MS').mean(),2)
dgs30 = pd.DataFrame(dgs30, index = dgs30.index, columns = ['dgs30'])

In [211]:
# U.S. / Euro Foreign Exchange Rate (DEXUSEU)
dexuseu = fred.get_series('dexuseu', start, end)
dexuseu = round(dexuseu.resample('MS').mean(),2)
dexuseu = pd.DataFrame(dexuseu, index = dexuseu.index, columns = ['dexuseu'])

In [213]:
# China / U.S. Foreign Exchange Rate (DEXCHUS)
dexchus = fred.get_series('dexchus', start, end)
dexchus = round(dexchus.resample('MS').mean(),2)
dexchus = pd.DataFrame(dexchus, index = dexchus.index, columns = ['dexchus'])

In [219]:
# U.S. / U.K. Foreign Exchange Rate (DEXUSUK)
dexusuk = fred.get_series('dexusuk', start, end)
dexusuk = round(dexusuk.resample('MS').mean(),2)
dexusuk = pd.DataFrame(dexusuk, index = dexusuk.index, columns = ['dexusuk'])

In [221]:
# Singapore / U.S. Foreign Exchange Rate (DEXSIUS)
dexsius = fred.get_series('dexsius', start, end)
dexsius = round(dexsius.resample('MS').mean(),2)
dexsius = pd.DataFrame(dexsius, index = dexsius.index, columns = ['dexsius'])

In [223]:
# NASDAQ Composite Index (NASDAQCOM)
nasdaqcom = fred.get_series('nasdaqcom', start, end)
nasdaqcom = round(nasdaqcom.resample('MS').mean(),2)
nasdaqcom = pd.DataFrame(nasdaqcom, index = nasdaqcom.index, columns = ['nasdaqcom'])

In [226]:
# Wilshire US Real Estate Investment Trust Total Market Index (Wilshire US REIT)(WILLREITIND)
willreitind = fred.get_series('willreitind', start, end)
willreitind = round(willreitind.resample('MS').mean(),2)
willreitind = pd.DataFrame(willreitind, index = willreitind.index, columns = ['willreitind'])

In [228]:
# Wilshire US Real Estate Securities Price Index (Wilshire US RESI) (WILLRESIPR)
willresipr = fred.get_series('willresipr', start, end)
willresipr = round(willresipr.resample('MS').mean(),2)
willresipr = pd.DataFrame(willresipr, index = willresipr.index, columns = ['willresipr'])

In [232]:
# Wilshire US Large-Cap Total Market Index (WILLLRGCAP)
willlrgcap = fred.get_series('willlrgcap', start, end)
willlrgcap = round(willlrgcap.resample('MS').mean(),2)
willlrgcap = pd.DataFrame(willlrgcap, index = willlrgcap.index, columns = ['willlrgcap'])

In [235]:
# Wilshire US Small-Cap Total Market Index (WILLSMLCAP)
willsmlcap = fred.get_series('willsmlcap', start, end)
willsmlcap = round(willsmlcap.resample('MS').mean(),2)
willsmlcap = pd.DataFrame(willsmlcap, index = willsmlcap.index, columns = ['willsmlcap'])

In [237]:
# CBOE Volatility Index: VIX (VIXCLS)
vixcls = fred.get_series('vixcls', start, end)
vixcls = round(vixcls.resample('MS').mean(),2)
vixcls = pd.DataFrame(vixcls, index = vixcls.index, columns = ['vixcls'])

In [242]:
# CBOE DJIA Volatility Index (VXDCLS)
vxdcls = fred.get_series('vxdcls', start, end)
vxdcls = round(vxdcls.resample('MS').mean(),2)
vxdcls = pd.DataFrame(vxdcls, index = vxdcls.index, columns = ['vxdcls'])

In [244]:
# St. Louis Fed Financial Stress Index (STLFSI)
stlfsi = fred.get_series('stlfsi', start, end)
stlfsi = round(stlfsi.resample('MS').mean(),2)
stlfsi = pd.DataFrame(stlfsi, index = stlfsi.index, columns = ['stlfsi'])

In [246]:
# Chicago Fed National Financial Conditions Index (NFCI)
nfci = fred.get_series('nfci', start, end)
nfci = round(nfci.resample('MS').mean(),2)
nfci = pd.DataFrame(nfci, index = nfci.index, columns = ['nfci'])

In [248]:
# Borrowings, All Commercial Banks (H8B3094NCBA)
h8b3094ncba = fred.get_series('h8b3094ncba', start, end)
h8b3094ncba = round(h8b3094ncba.resample('MS').mean(),2)
h8b3094ncba = pd.DataFrame(h8b3094ncba, index = h8b3094ncba.index, columns = ['h8b3094ncba'])

In [250]:
# Bank Credit, Domestically Chartered Commercial Banks (BC0DCBW027NBOG)
bc0dcbw027nbog = fred.get_series('bc0dcbw027nbog', start, end)
bc0dcbw027nbog = round(bc0dcbw027nbog.resample('MS').mean(),2)
bc0dcbw027nbog = pd.DataFrame(bc0dcbw027nbog, index = bc0dcbw027nbog.index, columns = ['bc0dcbw027nbog'])

In [252]:
# Cash Assets, Domestically Chartered Commercial Banks (CASDCBW027NBOG)
casdcbw027nbog = fred.get_series('casdcbw027nbog', start, end)
casdcbw027nbog = round(casdcbw027nbog.resample('MS').mean(),2)
casdcbw027nbog = pd.DataFrame(casdcbw027nbog, index = casdcbw027nbog.index, columns = ['casdcbw027nbog'])

In [254]:
# Commercial and Industrial Loans, Domestically Chartered Commercial Banks (CILDCBW027NBOG)
cildcbw027nbog = fred.get_series('cildcbw027nbog', start, end)
cildcbw027nbog = round(cildcbw027nbog.resample('MS').mean(),2)
cildcbw027nbog = pd.DataFrame(cildcbw027nbog, index = cildcbw027nbog.index, columns = ['cildcbw027nbog'])

In [256]:
# Deposits, Domestically Chartered Commercial Banks (DPSDCBW027NBOG)
dpsdcbw027nbog = fred.get_series('dpsdcbw027nbog', start, end)
dpsdcbw027nbog = round(dpsdcbw027nbog.resample('MS').mean(),2)
dpsdcbw027nbog = pd.DataFrame(dpsdcbw027nbog, index = dpsdcbw027nbog.index, columns = ['dpsdcbw027nbog'])

In [258]:
# Real Estate Loans: Residential Real Estate Loans: Revolving Home Equity Loans, Domestically Chartered Commercial Banks (RHEDCBW027NBOG)
rhedcbw027nbog = fred.get_series('rhedcbw027nbog', start, end)
rhedcbw027nbog = round(rhedcbw027nbog.resample('MS').mean(),2)
rhedcbw027nbog = pd.DataFrame(rhedcbw027nbog, index = rhedcbw027nbog.index, columns = ['rhedcbw027nbog'])

In [276]:
# Consumer Loans, All Commercial Banks (CLSACBW027NBOG)
clsacbw027nbog = fred.get_series('clsacbw027nbog', start, end)
clsacbw027nbog = round(clsacbw027nbog.resample('MS').mean(),2)
clsacbw027nbog = pd.DataFrame(clsacbw027nbog, index = clsacbw027nbog.index, columns = ['clsacbw027nbog'])

In [267]:
# Deposits, All Commercial Banks (DPSACBW027NBOG)
dpsacbw027nbog = fred.get_series('dpsacbw027nbog', start, end)
dpsacbw027nbog = round(dpsacbw027nbog.resample('MS').mean(),2)
dpsacbw027nbog = pd.DataFrame(dpsacbw027nbog, index = dpsacbw027nbog.index, columns = ['dpsacbw027nbog'])

In [269]:
# Real Estate Loans, All Commercial Banks (RELACBW027NBOG)
relacbw027nbog = fred.get_series('relacbw027nbog', start, end)
relacbw027nbog = round(relacbw027nbog.resample('MS').mean(),2)
relacbw027nbog = pd.DataFrame(relacbw027nbog, index = relacbw027nbog.index, columns = ['relacbw027nbog'])

In [274]:
# Real Estate Loans: Residential Real Estate Loans: Revolving Home Equity Loans, All Commercial Banks (RHEACBW027NBOG)
rheacbw027nbog = fred.get_series('rheacbw027nbog', start, end)
rheacbw027nbog = round(rheacbw027nbog.resample('MS').mean(),2)
rheacbw027nbog = pd.DataFrame(rheacbw027nbog, index = rheacbw027nbog.index, columns = ['rheacbw027nbog'])

In [277]:
# Total Assets, All Commercial Banks (TLAACBW027NBOG)
tlaacbw027nbog = fred.get_series('tlaacbw027nbog', start, end)
tlaacbw027nbog = round(tlaacbw027nbog.resample('MS').mean(),2)
tlaacbw027nbog = pd.DataFrame(tlaacbw027nbog, index = tlaacbw027nbog.index, columns = ['tlaacbw027nbog'])

In [278]:
# Total Liabilities, All Commercial Banks (TLBACBW027NBOG)
tlbacbw027nbog = fred.get_series('tlbacbw027nbog', start, end)
tlbacbw027nbog = round(tlbacbw027nbog.resample('MS').mean(),2)
tlbacbw027nbog = pd.DataFrame(tlbacbw027nbog, index = tlbacbw027nbog.index, columns = ['tlbacbw027nbog'])

In [281]:
# Commercial and Industrial Loans, All Commercial Banks (TOTCI)
totci = fred.get_series('totci', start, end)
totci = round(totci.resample('MS').mean(),2)
totci = pd.DataFrame(totci, index = totci.index, columns = ['totci'])

In [283]:
# Economic Policy Uncertainty Index for United States (USEPUINDXD)
usepuindxd = fred.get_series('usepuindxd', start, end)
usepuindxd = round(usepuindxd.resample('MS').mean(),2)
usepuindxd = pd.DataFrame(usepuindxd, index = usepuindxd.index, columns = ['usepuindxd'])

In [285]:
# Equity Market-related Economic Uncertainty Index (WLEMUINDXD)
wlemuindxd = fred.get_series('wlemuindxd', start, end)
wlemuindxd = round(wlemuindxd.resample('MS').mean(),2)
wlemuindxd = pd.DataFrame(wlemuindxd, index = wlemuindxd.index, columns = ['wlemuindxd'])

In [288]:
# Equity Market Volatility Tracker: Financial Crises (EMVFINCRISES)
emvfincrises = fred.get_series('emvfincrises', start, end)
emvfincrises = round(emvfincrises.resample('MS').mean(),2)
emvfincrises = pd.DataFrame(emvfincrises, index = emvfincrises.index, columns = ['emvfincrises'])

In [290]:
# Price Pressures Measure (STLPPM)
stlppm = fred.get_series('stlppm', start, end)
stlppm = round(stlppm.resample('MS').mean(),2)
stlppm = pd.DataFrame(stlppm, index = stlppm.index, columns = ['stlppm'])

In [303]:
rates_list = [usd1mtd156n, mortgage30us, mortgage15us, dcpf3m, cpn3m, dcpf1m, aaa, baa, dff, flexsc, t10y2y, t10y3m, tedrate, t10yff, t5yff, tb6smffm, usd3mtd156n, usd12md156n, 
              eur1wkd156n, mprime, tb3ms, tb1yr, dtb6, dgs10, dfii10, dgs3mo, dgs30, dexuseu, dexchus, dexusuk, dexsius, nasdaqcom, willreitind, willlrgcap, willsmlcap, vxdcls, 
              stlfsi, nfci, h8b3094ncba, bc0dcbw027nbog, casdcbw027nbog, cildcbw027nbog, dpsdcbw027nbog, rhedcbw027nbog, clsacbw027nbog, dpsacbw027nbog, relacbw027nbog, rheacbw027nbog, 
              tlaacbw027nbog, tlbacbw027nbog, totci, usepuindxd, wlemuindxd, emvfincrises, stlppm]
rates_df = pd.concat(rates_list, axis = 1)
rates_df.head(5)

,usd1mtd156n,mortgage30us,mortgage15us,dcpf3m,cpn3m,dcpf1m,aaa,baa,dff,flexsc,...,dpsacbw027nbog,relacbw027nbog,rheacbw027nbog,tlaacbw027nbog,tlbacbw027nbog,totci,usepuindxd,wlemuindxd,emvfincrises,stlppm
2000-02-01,5.89,8.33,7.93,5.90,5.87,5.78,7.68,8.29,5.73,5.85,...,3490.25,1486.01,105.51,5638.25,5157.79,1016.61,50.60,45.27,0.35,0.30
2000-03-01,6.05,8.24,7.83,6.03,6.00,5.94,7.68,8.37,5.85,5.93,...,3521.57,1510.16,107.31,5685.93,5198.06,1024.58,64.99,46.42,0.90,0.19
2000-04-01,6.15,8.15,7.80,6.15,6.11,6.03,7.64,8.40,6.02,6.15,...,3584.93,1531.49,110.75,5747.53,5262.45,1033.44,59.76,88.46,1.04,0.65
2000-05-01,6.54,8.51,8.18,6.57,6.54,6.41,7.99,8.90,6.27,6.30,...,3563.07,1559.57,113.35,5821.97,5336.37,1051.14,77.52,55.96,0.63,0.30
2000-06-01,6.65,8.29,7.99,6.59,6.57,6.53,7.67,8.48,6.53,6.62,...,3581.19,1578.90,114.16,5841.58,5345.38,1062.59,67.24,74.91,0.52,0.46
